In [165]:
import argparse
import os

import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import cifar10
from keras.layers import (Activation, Conv3D, Dense, Dropout, Flatten,
                          MaxPooling3D)
from keras.layers.advanced_activations import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import numpy as np
import cv2

In [166]:
batch = 15
epoch = 50
#videos = "/Users/gautamkarnataki/Downloads/train_val_videos/"
videos = "/Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3"
nclass = 19
output = "/Users/gautamkarnataki/MIDS/w251/project/3dcnn/3DCNN/3dcnnresult"
color = True
skip = True
depth = 10

In [167]:
class Videoto3D:

    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth

    def video3d(self, filename, color=False, skip=True):
        print('filename from videocapture = ', filename)
        cap = cv2.VideoCapture(filename)
        nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        if skip:
            frames = [x * nframe / self.depth for x in range(self.depth)]
        else:
            frames = [x for x in range(self.depth)]
        framearray = []

        for i in range(self.depth):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frames[i])
            ret, frame = cap.read()
            frame = cv2.resize(frame, (self.height, self.width))
            if color:
                framearray.append(frame)
            else:
                framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

        cap.release()
        return np.array(framearray)


    def get_UCF_classname(self, filename):
        string = filename[filename.find('-',3)+9:]
        #print("class =========>", string[:string.find('_')])
        return string[:string.find('_')]

In [168]:
def loaddata(video_dir, vid3d, nclass, result_dir, color=False, skip=True):
    print('Inside load data -------------------------------')
    files = os.listdir(video_dir)
    X = []
    labels = []
    labellist = []

    pbar = tqdm(total=len(files))

    for filename in files:
        pbar.update(1)
        if filename == '.DS_Store':
            continue
        name = os.path.join(video_dir, filename)
        label = vid3d.get_UCF_classname(filename)
        if label not in labellist:
            if len(labellist) >= nclass:
                continue
            labellist.append(label)
        labels.append(label)
        X.append(vid3d.video3d(name, color=color, skip=skip))

    pbar.close()
    print("result_dir==============>", result_dir)
    with open(os.path.join(result_dir, 'classes.txt'), 'w') as fp:
        for i in range(len(labellist)):
            print("label ============================= ", labellist[i])
            fp.write('{}\n'.format(labellist[i]))

    for num, label in enumerate(labellist):
        for i in range(len(labels)):
            if label == labels[i]:
                labels[i] = num
    if color:
        return np.array(X).transpose((0, 2, 3, 4, 1)), labels
    else:
        return np.array(X).transpose((0, 2, 3, 1)), labels

In [169]:
img_rows, img_cols, frames = 32, 32, depth
channel = 3 if color else 1
#fname_npz = 'dataset_{}_{}_{}.npz'.format(nclass, depth, skip)
vid3d = Videoto3D(img_rows, img_cols, frames)
nb_classes = nclass
#print("fname_npz ==========>", fname_npz)
if os.path.exists(fname_npz):
#    loadeddata = np.load(fname_npz)
#    X, Y = loadeddata["X"], loadeddata["Y"]
#else:
    x, y = loaddata(videos, vid3d, nclass, output, color, skip)
    X = x.reshape((x.shape[0], img_rows, img_cols, frames, channel))
    Y = np_utils.to_categorical(y, nb_classes)
    X = X.astype('float32')
    #np.savez(fname_npz, X=X, Y=Y)
    print('Saved dataset to dataset.npz.')
print('X_shape:{}\nY_shape:{}'.format(X.shape, Y.shape))

  1%|▏         | 2/142 [00:00<00:09, 14.48it/s]

Inside load data -------------------------------
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2008_05_12a_scene29-camera1_DRESS_v0_c5_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2006_10_10_scene6-camera1_CONFLICT_v0_c7_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2008_01_18_scene15-camera1_DEPRESS_v0_c0_train.mov


  4%|▍         | 6/142 [00:00<00:10, 13.23it/s]

filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2006_10_10_scene7-camera1_DOCTOR_v1_c6_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2008_05_12b_scene1-camera1_CHAT_v2_c3_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2007_05_24_scene10-camera1_COURT_v0_c8_train.mov


  6%|▌         | 8/142 [00:00<00:11, 12.03it/s]

...

filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2006_10_10_scene5-camera1_CHURCH_v1_c4_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2008_01_11_scene9-camera1_AGAIN_v0_c0_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2006_10_10_scene2-camera1_AGAIN_v0_c9_train.mov


100%|██████████| 142/142 [00:12<00:00, 11.51it/s]

filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2008_08_04_scene45-camera1_CHEAT_v0_c3_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2008_05_12a_scene8-camera1_AGAIN_v0_c2_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3/ASL_2008_01_11_scene81-camera1_COAT_v0_c0_train.mov


100%|██████████| 142/142 [00:12<00:00, 11.18it/s]

result_dir==============> /Users/gautamkarnataki/MIDS/w251/project/3dcnn/3DCNN/3dcnnresult
label =============================  DRESS
label =============================  CONFLICT
label =============================  DEPRESS
label =============================  DOCTOR
label =============================  CHAT
label =============================  COURT
label =============================  AGAIN
label =============================  ENOUGH
label =============================  BEHAVIOR
label =============================  COAT
label =============================  CAN
label =============================  CHEAP
label =============================  DEPOSIT
label =============================  WANT
label =============================  AWKWARD
label =============================  ALL
label =============================  CHURCH
label =============================  BASEBALL
label =============================  CHEAT
Saved dataset to dataset.npz.
X_shape:(142, 32, 32, 10, 3)
Y_shape:(142, 19)


In [171]:
# Define model
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(X.shape[1:]), border_mode='same'))
model.add(Activation('relu'))
model.add(Conv3D(32, kernel_size=(3, 3, 3), border_mode='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), border_mode='same'))
model.add(Dropout(0.25))

model.add(Conv3D(64, kernel_size=(3, 3, 3), border_mode='same'))
model.add(Activation('relu'))
model.add(Conv3D(64, kernel_size=(3, 3, 3), border_mode='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), border_mode='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss=categorical_crossentropy,optimizer=Adam(), metrics=['accuracy'])
model.summary()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=43)

for i in range(50):
    print('Y train =', Y_train[i])

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=batch,
                    epochs=epoch, verbose=1, shuffle=True)
model.evaluate(X_test, Y_test, verbose=0)
model_json = model.to_json()
if not os.path.isdir(output):
    os.makedirs(output)
with open(os.path.join(output, 'ucf101_3dcnnmodel.json'), 'w') as json_file:
    json_file.write(model_json)
model.save_weights(os.path.join(output, 'ucf101_3dcnnmodel.hd5'))

loss, acc = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', acc)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, kernel_size=(3, 3, 3), input_shape=(32, 32, 1..., padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, kernel_size=(3, 3, 3), padding="same")`
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(3, 3, 3), padding="same")`
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, kernel_size=(3, 3, 3), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv3D` call to the 

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_85 (Conv3D)           (None, 32, 32, 10, 32)    2624      
_________________________________________________________________
activation_85 (Activation)   (None, 32, 32, 10, 32)    0         
_________________________________________________________________
conv3d_86 (Conv3D)           (None, 32, 32, 10, 32)    27680     
_________________________________________________________________
activation_86 (Activation)   (None, 32, 32, 10, 32)    0         
_________________________________________________________________
max_pooling3d_43 (MaxPooling (None, 11, 11, 4, 32)     0         
_________________________________________________________________
dropout_64 (Dropout)         (None, 11, 11, 4, 32)     0         
_________________________________________________________________
conv3d_87 (Conv3D)           (None, 11, 11, 4, 64)   

113/113 [==============================] - 12s 102ms/step - loss: 2.7447 - accuracy: 0.2478 - val_loss: 3.0926 - val_accuracy: 0.1724
Epoch 17/50
113/113 [==============================] - 11s 97ms/step - loss: 2.6306 - accuracy: 0.2655 - val_loss: 3.0560 - val_accuracy: 0.1724
Epoch 18/50
113/113 [==============================] - 11s 96ms/step - loss: 2.5225 - accuracy: 0.3097 - val_loss: 3.0867 - val_accuracy: 0.1379
Epoch 19/50
113/113 [==============================] - 11s 95ms/step - loss: 2.5857 - accuracy: 0.2389 - val_loss: 3.1152 - val_accuracy: 0.1034
Epoch 20/50
113/113 [==============================] - 11s 94ms/step - loss: 2.4895 - accuracy: 0.2743 - val_loss: 3.0466 - val_accuracy: 0.1379
Epoch 21/50
113/113 [==============================] - 14s 123ms/step - loss: 2.3750 - accuracy: 0.2478 - val_loss: 2.9539 - val_accuracy: 0.1379
Epoch 22/50
113/113 [==============================] - 11s 101ms/step - loss: 2.3221 - accuracy: 0.2920 - val_loss: 2.9376 - val_accuracy: 0